In [ ]:
import numpy as np


In [ ]:
#Hidden states
states = ['Rainy', 'Sunny']
n_states = len(states)

In [ ]:
#Observable states
observations = ['walk', 'shop', 'clean']
n_observations = len(observations)